## Select-For Structure from Motion (SfM) workflow
----------------------

The following workflow demonstrates the full SfM process using open source libraries on a recently collected over a a forest plantation managed by Select-For. The aim of the collecting and processing the data is to provide a DSM, classified point cloud and Canopy Height Model (CHM). SfM is not really optimal for CHM generation, but can  give a good enough model with a combination of automated and manual routines. 

This dataset consists of 150 RGB image captures using the SONY A-6000 sensor aboard the Bramor PPX fixed wing platform.

This is a subset of the original dataset (~5000 images) as processing would be too lengthy for the demonstration. 

Survey conditions were good with diffuse light for the duration of the flight.

GPS is corrected using the Ordnance survey network.

**Dependencies:**

You will need to install this python module to run bash commands without constant exclamation marks

pip install bash_kernel

python -m bash_kernel.install

Firstly install the MicMac lib.

https://micmac.ensg.eu/index.php/Accueil

The SfM scripts are here and must be added to your system path to work.

https://github.com/Ciaran1981/Sfm


QGIS 3 offers a versatile platform for this purpose for the final datasets, so please ensure this is installed. Once installed, install the HCMGIS plugin to use various web-based base-layers for visualisation purposes. 

https://www.osgeo.org/projects/qgis/

Cloud compare or meshlab provide a useful tool to visualise point cloud results and can be found through the links below.

https://www.danielgm.net/cc/

http://www.meshlab.net/

Part of the final section of this workflow uses PDAL to provide an initial classification. It is likely best to install this in a separate anaconda environment.  

https://pdal.io/

Installation of pdal is most easily achieved via anaconda, so remember to change kernel at the appropriate point.

**Assumed knowledge:**

Basic GIS, image processing, remote sensing and command line use

Remember to query function args open a new cell (the plus icon) and write:

```bash
Orientation.sh -help
```
**Though each of these commands have a check list and a proceed question first**

The data is temporarily found in this google drive folder until QinetiQ find somewhere to store it.

https://drive.google.com/drive/folders/1CLjXTn5iT4JuGI-LrB1aErKh9I-X8-VG?usp=sharing

The file required is **Forest.zip** - extract this somewhere appropriate and change dir to this folder.

This contains: 

- a collection of JPG images 
- plot2.csv (the main GPS log)
- calib.csv (a lens caslibration subset - optional)


In [ ]:
# insert you working directory
cd /path/wrkdir

### Command Breakdown

It is usually best practice to evaluate bundle adjustment results etc., hence processing individually will aid in checking, the sparse point cloud from which the eventual dense one is derived.

For this demo, the rgb_sfm using the PIMs (per-image-matching) and forest parameters will suffice.

The workflow could be split up into individual commands for each stage also. 

### Features and Orienation

Here we detect features and orient images. 

As the dataset GPS log has been base-station corrected, we can set an estimation (conservative still!) of 1 meter. 

At the end the sparse point cloud will open in meshlab (remember to install this!) for evaluation).

I have commented out the calibration subset as this not manadatory but illustrates the underlying process and may be a good option depending on time/dataset size constraints. Otherwize the MicMac Martini command is used to intialise the image block without radial distortion, which is then fed to the global orientation. 

The salient commands internal to the script are as follows, where input variables are prefixed with $ (I have excluded conditional statements and admin cmds to make this clear- look at the script for complete detail):

```bash
# Generate a GPS and photo index
mm3d OriConvert OriTxtInFile ${CSV} RAWGNSS_N ChSys=DegreeWGS84@SysUTM.xml MTD1=1 NameCple=FileImagesNeighbour.xml CalcV=1 

# SIFT-based feature detection and matching
mm3d Tapioca File FileImagesNeighbour.xml -1  @SFS

# Tie point reduction to an evenly spaced set
mm3d Schnaps .*${EXTENSION} MoveBadImgs=1

# Initialise orientation without lense distortion
mm3d Martini .*${EXTENSION}

# Relative image orientation
mm3d Tapas ${CALIB} .*${EXTENSION} Out=Arbitrary inOri=Martini | tee ${CALIB}RelBundle.txt

# Change coordinate system for GPS aided bundle adjustment
mm3d CenterBascule .*${EXTENSION} Arbitrary RAWGNSS_N Ground_Init_RTL

#Bundle adjust using both camera positions and tie points (number in EmGPS option is the quality estimate of the GNSS data in meters)
mm3d Campari .*${EXTENSION} Ground_Init_RTL Ground_UTM EmGPS=[RAWGNSS_N,1] AllFree=1  | tee ${CALIB}GnssBundle.txt

#Generate a sparse pointcloud for inspection
mm3d AperiCloud .*${EXTENSION} Ground_UTM
```

In [ ]:
# or include the calib-subset
# uncomment the line to run
# Orientation.sh -e tif -u "30 +north" -c Fraser -t "plot2.csv" -s "sub.csv" 

In [1]:
Orientation.sh -e tif -u "30 +north" -c Fraser -t "plot2.csv" 


### Dense matching & ortho-photos

For this dataset the PIMs and forest modes are used and given the diffuse lighting no radiometric equalisation will be carried out.


The main process is carried out via the bash script below, which itself contains the following sequnce of MicMac commands which are critical to the outputs being produced. **Some other things are done to move files around so please look at the script for details of everything**. Input variables are prefixed with $. 

```bash

#Calculate dense cloud
mm3d PIMs $Algorithm .*$EXTENSION Ground_UTM DefCor=0 ZoomF=$ZoomF ZReg=$zreg Masq3D=AperiCloud_Ground_UTM.ply.xml

#Generate DSM and ortho photos
mm3d PIMs2MNT $Algorithm DoMnt=1 DoOrtho=1

#Generate the Mosaic
mm3d Tawny PIMs-ORTHO/ RadiomEgal=0 Out=Orthophotomosaic.tif

#Merge the mosaic tiles if they exist 
mm3d ConvertIm PIMs-ORTHO/Orthophotomosaic.tif Out=OUTPUT/OrthFinal.tif
cp PIMs-ORTHO/Orthophotomosaic.tfw OUTPUT/OrthFinal.tfw

#Georeference the image (same is done for DSM)
gdal_edit.py -a_srs "+proj=utm +zone=$UTM +ellps=WGS84 +datum=WGS84 +units=m +no_defs" OUTPUT/OrthFinal.tif

#Generate point cloud
mm3d Nuage2Ply PIMs-TmpBasc/PIMs-Merged.xml Attr=PIMs-ORTHO/Orthophotomosaic.tif Out=OUTPUT/pointcloud.ply
```

In [ ]:
dense_cloud.sh -e tif -a Forest -m PIMs -u 30 +north -i 0

### Outputs

We can view the results in QGIS for the imagery and meshlab/cloudcompare for the point cloud 

mm3d Nuage2Ply PIMs-TmpBasc/PIMs-Merged.xml Attr=PIMs-ORTHO/Orthophotomosaic.tif Out=OUTPUT/pointcloud.ply

### Point cloud classification

As SfM is modelled approach, it does not lend itself to point cloud classification in the same way LiDAR (a direct measurment of sorts) does.

Reasonable results may still be obtained, however with a combination of automated routines and manual editing.

For the automated part, the pdal libray is used to provided an initial filtered point cloud from which a manual tidy-up can be performed. 

**activate pdal first via command line (NOT the notebook)**

In [ ]:
conda activate pdal

In [ ]:
pdal translate -i OUTPUT/psm.ply -o OUTPUT/psm.laz

Now for the initial classification of ground points using the progressive morphological filter.

In [ ]:
pdal ground -i OUTPUT/psm.laz -o OUTPUT/classif.laz

Finally we retain only the ground points prior to manual editing.

In [ ]:
pdal translate -i classif.laz smrf range --filters.range.limits="Classification[2:2]" -o grnd.laz

### Manual tidying

From here we have to manually edit the point cloud using cloud compare, for which there are tutorials online.

The procedure follows the process described here:

https://www.cloudcompare.org/doc/wiki/index.php?title=Interactive_Segmentation_Tool

Through multiple (manual) iterations you can crop the areas you wish to retain/delete until a satisfactory result is obtained. 

Unfortunately, to obtain better results manual labour is the only way.

**Probably best not to embark on this task unless you wish/need to know how to do this manually.**